# WEEK 1
## STEP 1
I downloaded the dataset from kaggle using *download_dataset.py* file and chose 5 High CAP companies - **AAPL**, **AMZN**,**GOOGL**,**MSFT** and **TSLA**. I added the files into a directory named `stocks`.
Imported `pandas` library to make multiindexed csv out of the `.us.txt` files.
Imported `glob` library to locate and read all the files with `.us.txt` suffix.

In [285]:
import pandas as pd
import glob

Sorted the companies alphabetically to maintain a order.

In [286]:
company_files = sorted(glob.glob('stocks/*.us.txt'))


In [287]:
company_df_list=[]
company_name_list=[]

### Inside a `for` loop
* From the relative file path, extracted out company name using indexing and capitalized it and stored it in a variable name called `ticker`.
* Made a dataframe for each individual comapny.
* Converted the `'Date'` column of string type into the pandas datetime object using `parse_dates` parameter. 
* Added the new column `'Ticker'` and assigned it `ticker` for all rows of each company.
* Appended each dataframe named `company` to `company_df_list`.

In [288]:
for file in company_files:
    ticker=file.split('.')[0][7:].upper()
    company_name_list.append(ticker)
    company=pd.read_csv(file, parse_dates=['Date'])
    company['Ticker']=ticker
    company_df_list.append(company)

Used `concat` function to convert the list to dataframe.

In [289]:
df=pd.concat(company_df_list)

Multi-Indexed the dataframe with `Ticker` as first and `Date` as second index. 

In [290]:
df.set_index(['Ticker', 'Date'], inplace=True)
# print(df.index.get_level_values('Date').dtype)

## STEP 2
Looking for rows having NaN values

In [291]:
missing_rows=[]

for index, row in df.iterrows():
    ticker=index[0]
    date=index[1]
    if row.isnull().any():
        missing_rows.append()
# print(missing_rows)

Conclusion - There are no missing values in the dataframe.

## Sorting the data
For each Ticker the rows are sorted in ascending order of the dates.

In [292]:
df.sort_index(level=['Ticker','Date'])

Open       High        Low      Close    Volume  \
Ticker Date                                                               
AAPL   1984-09-07    0.42388    0.42902    0.41874    0.42388  23220030   
       1984-09-10    0.42388    0.42516    0.41366    0.42134  18022532   
       1984-09-11    0.42516    0.43668    0.42516    0.42902  42498199   
       1984-09-12    0.42902    0.43157    0.41618    0.41618  37125801   
       1984-09-13    0.43927    0.44052    0.43927    0.43927  57822062   
...                      ...        ...        ...        ...       ...   
TSLA   2017-11-06  307.00000  307.50000  299.01000  302.78000   6482486   
       2017-11-07  301.02000  306.50000  300.03000  306.05000   5286320   
       2017-11-08  305.50000  306.89000  301.30000  304.31000   4725510   
       2017-11-09  302.50000  304.46000  296.30000  302.99000   5440335   
       2017-11-10  302.50000  308.36000  301.85000  302.99000   4621912   

                   OpenInt  
Ticker Date                 
AAPL   1984-09-07        0  
       1984-09-10        0  
       1984-09-11        0  
       1984-09-12        0  
       1984-09-13        0  
...                    ...  
TSLA   2017-11-06        0  
       2017-11-07        0  
       2017-11-08        0  
       2017-11-09        0  
       2017-11-10        0  

[26691 rows x 6 columns]

## Filtering the data
The date nearest to last 10 years was found and then the df was sliced there to remove older dates from each Ticker.

In [293]:
# df.loc[('GOOGL','2017-11-09')]
#df = df[df.index.get_level_values(1) >= '2007-11-09']

## STEP 3
Adding the columns

In [294]:
df['Daily Return']=0.0
df['7-day Moving Average']=0.0
df['30-day Moving Average']=0.0
df['Rolling Volatility (30d)']=0.0

for i in range(1,len(df)):
    current_closing = df.iloc[i,3]
    previous_closing = df.iloc[i-1,3]
    percentage_change= ((current_closing-previous_closing)/previous_closing)*100
    df.iloc[i,6]=percentage_change
    if (i>=6):
        av_closing_7=0
        for j in range(6):
            av_closing_7+=df.iloc[i-j,3]
        av_closing_7 /= 7
        df.iloc[i,7]=av_closing_7
    if (i>=29):
        av_closing_30=0
        stand_dev=0
        for j in range(29):
            av_closing_30+=df.iloc[i-j,3]
        av_closing_30 /= 30
        df.iloc[i,8]=av_closing_30
        for j in range(29):
            stand_dev+=abs(df.iloc[i-j,3]-av_closing_30)
        stand_dev /= 30
        df.iloc[i,9]=stand_dev


**Note: I added the columns and filled them with the corresponding data before filtering for newly added columns, without losing older (datewise) rows. Otherwise they would have had NaN values for some number of first rows due to moving average.**

In [ ]:
df = df[df.index.get_level_values(1) >= '2007-11-09']

11938


## STEP 4
The company with highest average return is `TSLA`. The highest average return is around 0.17%.


In [305]:
average_returns=[]
for company_name in company_name_list:
    average_returns.append((df.loc[company_name]['Daily Return'].sum())/len(df.loc[company_name]))
# print(company_name_list[average_returns.index(max(average_returns))])
# print(max(average_returns))

The company with highest monthly (30d) volatility is `GOOGl` on `2015-08-07`. The highest volatility is around `56.65`.

In [309]:
max_volatility=df['Rolling Volatility (30d)'].max()
max_idx=df['Rolling Volatility (30d)'].idxmax()
# print(max_idx[0])
print(max_idx[1])

2015-08-07 00:00:00
